### Import Data & Modules

In [ ]:
import pandas as pd
from pathlib import Path
import kagglehub
from sqlalchemy import create_engine
from openai import OpenAI
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Download all files
path = kagglehub.dataset_download("datafiniti/consumer-reviews-of-amazon-products")
csv_paths = list(Path(path).glob("*.csv"))


dfs = [pd.read_csv(p) for p in csv_paths]
df = pd.concat(dfs, ignore_index=True)

df = df.drop_duplicates(
    subset=['id', 'dateAdded', 'dateUpdated', 'reviews.text', 'reviews.title', 'reviews.username'],
    keep='first'
).reset_index(drop=True)

df = df.dropna(subset=["reviews.text", "reviews.title", "reviews.rating"])

engine = create_engine('sqlite:///data/reviews.db')
df.to_sql('reviews_table', engine, index=False, if_exists='replace')

print(df.shape)

### Model 1


#### Add Code here

### Model 2

In [ ]:
import random
rand_subset = random.sample(range(len(df)), 3000)

review_list = []
for i in rand_subset:

        prompt = "Text: " + df['reviews.text'].iloc[i]  + "  Title: " + df['reviews.title'].iloc[i]
        review_list.append(prompt)


review_text = "\n".join(review_list)

print(review_text)
print('NUMER OF TOKENS: ', len(review_text.split()))


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')



client = OpenAI(
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-4.1-nano"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    text = response.choices[0].message.content
    tokens_in = response.usage.prompt_tokens
    tokens_out = response.usage.completion_tokens
    tokens_total = response.usage.total_tokens
    
    return text, {"input": tokens_in, "output": tokens_out, "total": tokens_total}

In [ ]:

prompt = f"""
Determine 6 product topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas without numbering them.

Text sample: '''{review_text}'''
"""
text, usage = get_completion(prompt)
print("Response:", text)
print("Token usage:", usage)
print("Cost: $", float(usage['total'])/1000000 * 0.1)